In [78]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [79]:
theater_seat_organizer = gp.Model()

# The Parameters

In [141]:
number_of_customers_in_group = [9, 9, 9, 9]
customer_vaccination_status = [[0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0]]
number_of_seats = 80
non_vax_cost = 1

# Parameter Validation Logic

In [142]:
def isBinary(num):
    if(num == 1 or num == 0):
        return True
    else:
        return False


# Vaccine Records Should Match Customer Records
number_of_vax = count = sum( [ len(customers) for customers in customer_vaccination_status])

if(sum(number_of_customers_in_group) != number_of_vax):
    raise ValueError('Vaccine Records Do Not Match Customer Records')
    
# Vaccine Records Should be Formatted like Customer Records
for index, customer_ofGroup_vaxList in enumerate(customer_vaccination_status):
    if any(not isBinary(person) for person in customer_ofGroup_vaxList):
        raise ValueError('Invalid Arg: Person is either vaxed = 1 or unvaxxed = 0')
        
    if(len(customer_ofGroup_vaxList) != number_of_customers_in_group[index]):
        raise ValueError('Vaccine Records Are Not Formatted Correctly')



# Decision Variables

In [143]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [144]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [145]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [146]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")


In [147]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [148]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

In [149]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 166998 rows, 6698 columns and 353362 nonzeros
Model fingerprint: 0xc50149e8
Variable types: 0 continuous, 6698 integer (6698 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 36 (0.80s)
Loaded MIP start from previous solve with objective 36

Presolve removed 164765 rows and 3814 columns
Presolve time: 1.03s
Presolved: 2233 rows, 2884 columns, 66234 nonzeros
Variable types: 0 continuous, 2884 integer (2884 binary)

Explored 0 nodes (0 simplex iterations) in 1.88 seconds
Thread count was 16 (of 16 available processors)

Solution count 1: 36 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.

In [150]:
theater_seat_organizer.getConstrs()

[<gurobi.Constr at_most_one_person_at_seat0>,
 <gurobi.Constr at_most_one_person_at_seat1>,
 <gurobi.Constr at_most_one_person_at_seat2>,
 <gurobi.Constr at_most_one_person_at_seat3>,
 <gurobi.Constr at_most_one_person_at_seat4>,
 <gurobi.Constr at_most_one_person_at_seat5>,
 <gurobi.Constr at_most_one_person_at_seat6>,
 <gurobi.Constr at_most_one_person_at_seat7>,
 <gurobi.Constr at_most_one_person_at_seat8>,
 <gurobi.Constr at_most_one_person_at_seat9>,
 <gurobi.Constr at_most_one_person_at_seat10>,
 <gurobi.Constr at_most_one_person_at_seat11>,
 <gurobi.Constr at_most_one_person_at_seat12>,
 <gurobi.Constr at_most_one_person_at_seat13>,
 <gurobi.Constr at_most_one_person_at_seat14>,
 <gurobi.Constr at_most_one_person_at_seat15>,
 <gurobi.Constr at_most_one_person_at_seat16>,
 <gurobi.Constr at_most_one_person_at_seat17>,
 <gurobi.Constr at_most_one_person_at_seat18>,
 <gurobi.Constr at_most_one_person_at_seat19>,
 <gurobi.Constr at_most_one_person_at_seat20>,
 <gurobi.Constr at_most

In [151]:
theater_seat_organizer.getObjective()

<gurobi.LinExpr: -4.0 + person0_inGroup0_atSeat0 + person0_inGroup0_atSeat1 + person0_inGroup0_atSeat2 + person0_inGroup0_atSeat3 + person0_inGroup0_atSeat4 + person0_inGroup0_atSeat5 + person0_inGroup0_atSeat6 + person0_inGroup0_atSeat7 + person0_inGroup0_atSeat8 + person0_inGroup0_atSeat9 + person0_inGroup0_atSeat10 + person0_inGroup0_atSeat11 + person0_inGroup0_atSeat12 + person0_inGroup0_atSeat13 + person0_inGroup0_atSeat14 + person0_inGroup0_atSeat15 + person0_inGroup0_atSeat16 + person0_inGroup0_atSeat17 + person0_inGroup0_atSeat18 + person0_inGroup0_atSeat19 + person0_inGroup0_atSeat20 + person0_inGroup0_atSeat21 + person0_inGroup0_atSeat22 + person0_inGroup0_atSeat23 + person0_inGroup0_atSeat24 + person0_inGroup0_atSeat25 + person0_inGroup0_atSeat26 + person0_inGroup0_atSeat27 + person0_inGroup0_atSeat28 + person0_inGroup0_atSeat29 + person0_inGroup0_atSeat30 + person0_inGroup0_atSeat31 + person0_inGroup0_atSeat32 + person0_inGroup0_atSeat33 + person0_inGroup0_atSeat34 + person

In [152]:
theater_seat_organizer.getVars()

[<gurobi.Var person0_inGroup0_atSeat0 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat1 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat2 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat3 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat4 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat5 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat6 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat7 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat8 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat9 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat10 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat11 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat12 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat13 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat14 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat15 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat16 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat17 (value 0.0)>,
 <gurobi.Var person0_inGroup0_atSeat18 (value 0.0)>,
 <g

In [153]:
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")

Seat0: <empty>
Seat1: person1 fromGroup0
Seat2: <empty>
Seat3: <empty>
Seat4: person0 fromGroup2
Seat5: <empty>
Seat6: <empty>
Seat7: person0 fromGroup1
Seat8: <empty>
Seat9: <empty>
Seat10: <empty>
Seat11: <empty>
Seat12: <empty>
Seat13: <empty>
Seat14: <empty>
Seat15: person2 fromGroup1
Seat16: <empty>
Seat17: <empty>
Seat18: <empty>
Seat19: <empty>
Seat20: <empty>
Seat21: <empty>
Seat22: <empty>
Seat23: person3 fromGroup1
Seat24: <empty>
Seat25: person2 fromGroup0
Seat26: <empty>
Seat27: <empty>
Seat28: person7 fromGroup0
Seat29: <empty>
Seat30: person7 fromGroup1
Seat31: person8 fromGroup1
Seat32: person1 fromGroup1
Seat33: <empty>
Seat34: person4 fromGroup2
Seat35: person7 fromGroup2
Seat36: person1 fromGroup2
Seat37: <empty>
Seat38: person8 fromGroup2
Seat39: <empty>
Seat40: person5 fromGroup2
Seat41: person2 fromGroup2
Seat42: <empty>
Seat43: <empty>
Seat44: <empty>
Seat45: person6 fromGroup1
Seat46: <empty>
Seat47: <empty>
Seat48: <empty>
Seat49: person8 fromGroup3
Seat50: pers

{0: <gurobi.Var group0_admitted (value 1.0)>, 1: <gurobi.Var group1_admitted (value 1.0)>, 2: <gurobi.Var group2_admitted (value 1.0)>, 3: <gurobi.Var group3_admitted (value 1.0)>}
